In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect, func
from config import *

In [2]:
# create an engine to postgresql db
user = config['psql_user']
password = config['psql_pwd']
host = 'localhost'
port = '5432'
db = config['psql_db']
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(user, password, host, port, db)

# The return value of create_engine() is our connection object
engine = sqlalchemy.create_engine(url, client_encoding='utf8')

In [3]:
query = 'SELECT * FROM vpositions'
df_pos = pd.read_sql_query(query, engine)
df = df_pos.loc[(df_pos['cshares'] > 0) & (df_pos['file_date'] <='2017-06-30')].sort_values(by=['file_date','ticker'])
df['price2'] = 0
df

,file_date,name,ticker,mval,cmval,shares,cshares,price,price2
362,2013-09-30,BANK OF NEW YORK MELLON CORP,BK,7.442690e+08,5.300500e+07,24652836.0,8807.0,30.189995,0
670,2013-09-30,DAVITA HEALTHCARE PARTNERS I,DVA,1.789325e+09,-1.952300e+07,31446830.0,16472924.0,56.900012,0
567,2013-09-30,SUNCOR ENERGY INC NEW,SU,6.443960e+08,1.203750e+08,18009957.0,240500.0,35.779985,0
367,2013-09-30,US BANCORP DEL,USB,2.894119e+09,6.439400e+07,79117501.0,840200.0,36.580010,0
730,2013-09-30,VERISIGN INC,VRSN,5.578320e+08,7.115300e+07,10961520.0,64100.0,50.890023,0
112,2013-12-31,DAVITA HEALTHCARE PARTNERS I,DVA,2.310552e+09,5.212270e+08,36461294.0,5014464.0,63.369994,0
1,2013-12-31,GENERAL ELECTRIC CO,GE,2.967120e+08,2.826430e+08,10585502.0,9996602.0,28.030036,0
545,2013-12-31,US BANCORP DEL,USB,3.204565e+09,3.104460e+08,79320901.0,203400.0,40.400008,0
65,2013-12-31,U S G CORP,USG,9.901840e+08,5.022610e+08,34890174.0,17817982.0,28.380025,0
90,2013-12-31,WELLS FARGO & CO NEW,WFC,2.104100e+10,1.904400e+09,463458123.0,326500.0,45.399998,0


In [4]:
import os
import csv
import time
import requests
from bs4 import BeautifulSoup

In [5]:
# https://www.marketwatch.com/investing/stock/vrsn/historical?siteid=mktw&date=09%2F30%2F2013&x=0&y=0
base_url1 = "https://www.marketwatch.com/investing/stock/"
base_url2 = "/historical?siteid=mktw&date="

for index, row in df.iterrows():
    if row.file_date == '2017-06-30':
        yr = '2017'
        mth = '10'
        day = '02'
    elif row.file_date == '2016-09-30':
        yr = '2017'
        mth = '01'
        day = '03'
    else:
        yr = row.file_date[0:4]
        mth = row.file_date[5:7]
        day = row.file_date[8:10]
        if mth == "03":
            mth = "06"
            day = "30"
        elif mth == "06":
            mth = "09"
        elif mth == "09":
            mth = "12"
            day = "31"
        else: # mth == "12":
            mth = "03"
            yr = str(int(yr)+1)
    search_date = mth+"/"+day+"/"+yr
    
    #time.sleep(10)
    url= base_url1 + row.ticker + base_url2+search_date
    print(row.file_date, url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "lxml")
    try:
        img = soup.find("div",{"id":"quotedata"}).find("img")
        img_str = str(img).split('=')
        price_str = img_str[3].split('$')[0]
        real_price = price_str[::-1]
        
        df.loc[index, "price2"] = float(real_price)
    except:
        print(f'skipping {row.ticker} for file date: {row.file_date}')
        continue

2013-09-30 https://www.marketwatch.com/investing/stock/BK/historical?siteid=mktw&date=12/31/2013
2013-09-30 https://www.marketwatch.com/investing/stock/DVA/historical?siteid=mktw&date=12/31/2013
2013-09-30 https://www.marketwatch.com/investing/stock/SU/historical?siteid=mktw&date=12/31/2013
2013-09-30 https://www.marketwatch.com/investing/stock/USB/historical?siteid=mktw&date=12/31/2013
2013-09-30 https://www.marketwatch.com/investing/stock/VRSN/historical?siteid=mktw&date=12/31/2013
2013-12-31 https://www.marketwatch.com/investing/stock/DVA/historical?siteid=mktw&date=03/31/2014
2013-12-31 https://www.marketwatch.com/investing/stock/GE/historical?siteid=mktw&date=03/31/2014
2013-12-31 https://www.marketwatch.com/investing/stock/USB/historical?siteid=mktw&date=03/31/2014
2013-12-31 https://www.marketwatch.com/investing/stock/USG/historical?siteid=mktw&date=03/31/2014
2013-12-31 https://www.marketwatch.com/investing/stock/WFC/historical?siteid=mktw&date=03/31/2014
2013-12-31 https://www

In [6]:
df

,file_date,name,ticker,mval,cmval,shares,cshares,price,price2
362,2013-09-30,BANK OF NEW YORK MELLON CORP,BK,7.442690e+08,5.300500e+07,24652836.0,8807.0,30.189995,34.94
670,2013-09-30,DAVITA HEALTHCARE PARTNERS I,DVA,1.789325e+09,-1.952300e+07,31446830.0,16472924.0,56.900012,63.37
567,2013-09-30,SUNCOR ENERGY INC NEW,SU,6.443960e+08,1.203750e+08,18009957.0,240500.0,35.779985,35.05
367,2013-09-30,US BANCORP DEL,USB,2.894119e+09,6.439400e+07,79117501.0,840200.0,36.580010,40.40
730,2013-09-30,VERISIGN INC,VRSN,5.578320e+08,7.115300e+07,10961520.0,64100.0,50.890023,59.78
112,2013-12-31,DAVITA HEALTHCARE PARTNERS I,DVA,2.310552e+09,5.212270e+08,36461294.0,5014464.0,63.369994,68.85
1,2013-12-31,GENERAL ELECTRIC CO,GE,2.967120e+08,2.826430e+08,10585502.0,9996602.0,28.030036,25.89
545,2013-12-31,US BANCORP DEL,USB,3.204565e+09,3.104460e+08,79320901.0,203400.0,40.400008,42.86
65,2013-12-31,U S G CORP,USG,9.901840e+08,5.022610e+08,34890174.0,17817982.0,28.380025,32.72
90,2013-12-31,WELLS FARGO & CO NEW,WFC,2.104100e+10,1.904400e+09,463458123.0,326500.0,45.399998,49.74


In [7]:
root_path = os.getcwd()
output = os.path.join(root_path,"cshares.csv")
df.to_csv(output)

## Issues

* marketwatch historical price data is an image where the name is the reverse of the value
* splits can cause dramatic price differences